# MetaCast: Disease X Demonstration

MetaCast has been developed from the codebase used in [Grunnill et al. (2024)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011018). MetaCast's main feature is broad**CAST**ing epidemiological and ecological subpopulation models over multiple dimensions (axis) to form a **META**population model.

This notebook uses a disease X $S E I H R$ (Susceptible - Exposed - Infected - Hospitalised - Recovered) subpopulation model as a motivating example. In this notebook we will model possible scenarios an outbreak of disease X may present:
1. Risk-Structured: Single Dimension/Axis disease X model.
2. Seeding at X% prevalence: MultinomialSeeder
3. A Rapid Pathogen test: Discrete Event simulation within a 2 dimensional/axis disease X model.
4. New Vaccine: Multi Dimension/Axis disease X model with flows between some subpopulations.
5. New Vaccine's Efficacy is Uncertain: Latin Hypercube Sampling in simulation of a 2 dimensional/axis disease X model.

In all of these scenarios we will focus on Peak Hospitilisation and Total Hospitalisations as outputs of the simulated disease X outbreak.

## Suggested Reading: Metapopulation and Structured Population Models.

**Note** MetaCast has tried to keep to the mathematical notation seen in the first of the following:

Keeling, M. J., & Rohani, P. (2008). Metapopulations. In Modeling Infectious Diseases in Humans and Animals (pp. 237–240). Princeton University Press.

Keeling, M. J., & Rohani, P. (2008). Age-Structure: Childhood Infections. In Modeling Infectious Diseases in Humans and Animals (pp. 77–92). Pinston University Press.

## Suggested Reading: Performing Sensitivity Analyses.

Marino, S., Hogue, I. B., Ray, C. J., & Kirschner, D. E. (2008). A methodology for performing global uncertainty and sensitivity analysis in systems biology. In Journal of Theoretical Biology (Vol. 254, Issue 1, pp. 178–196). [https://doi.org/10.1016/j.jtbi.2008.04.011](https://doi.org/10.1016/j.jtbi.2008.04.011), [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2570191/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2570191/)

## Suggested Reading: Demonstrating a Scenario Analyses that can be Performed using this Package

`MetaCast` was developed from the code base that was previously used in:

Grunnill, M., Arino, J., McCarthy, Z., Bragazzi, N. L., Coudeville, L., Thommes, E., Amiche, A., Ghasemi, A., Bourouiba, L., Tofighi, M., Asgary, A., Baky-Haskuee, M., & Wu, J. (2024). Modelling Disease Mitigation at Mass Gatherings: A Case Study of COVID-19 at the 2022 FIFA World Cup. In E. H. Lau (Ed.), PLoS Computational Biology: Vol. January (Issue 1, p. e1011018). Public Library of Science. [https://doi.org/10.1371/JOURNAL.PCBI.1011018](https://doi.org/10.1371/JOURNAL.PCBI.1011018)

# 1. Risk-Structured: Single Dimension/Axis disease X model.

In this section we will be using our model with two groups, one at high risk of hospitalisation and another at a low risk (as shown in the equations below). The $r$ subscript denotes risk group, $p_{r}$ denotes the probability of becoming hospitalised for that risk group and $\lambda_{r}$ is the force of infection for that risk group. $\sigma^{-1}$ is the latency period. After $\gamma^{-1}$ days infectious individuals either recover ($1-p_{r}$) or are hospitalised ($p_{r}$). Hospitalised individuals recover after $\eta^{-1}$ days.
$$
    \frac{\delta S_{r}}{\delta t} = -\lambda_{r} S_{r}
$$
$$
    \frac{\delta E_{r}}{\delta t} = \lambda_{r} S_{r} - \sigma E_{r}
$$
$$
    \frac{\delta I_{r}}{\delta t} = \sigma E_{r} - \gamma I_{r}
$$
$$
    \frac{\delta H_{r}}{\delta t} = p_{r} \gamma I_{r} - \eta H_{r}
$$
$$
    \frac{\delta R_{r}}{\delta t} = (1-p_{r}) \gamma I_{r} + \eta H_{r}
$$

The force of infection ($\lambda_{r}$), shown below, for subpopulation $i$ is dependent on the number of infecteds in subpopulation $j$, the strength of interaction between subpopulation those subpopulations ($\rho_{ij}$) and the general transmission term for subpopulation $i$ ($\beta_{i}$).
$$
    \lambda_{i} = \beta_{i} \sum_{j} \rho_{ij} I_j
$$

## 1.1 MetaCaster setup

`MetaCast`'s main workhorse is the `MetaCaster` class. This needs to be imported.

In [1]:
from metacast import MetaCaster # Import MetaCast's main class MetaCaster.

### 1.1.1 Define subpopulation model.

The subpop_model function is your subpopulation model (see 1.) that `MetaCaster` will broadcast across the metapopulation generated by your dimensions (see 1.2).
The docstring below explains how the required arguments are used by `MetaCaster`.

In [2]:
def subpop_model(y, y_deltas, parameters, states_index, subpop_suffix, foi):
    """
    Calculate derivatives of variables in disease X's subpopulation model

    Parameters
    ----------
    y : numpy.Array
        An array of the state variables at this time point.
    y_deltas : numpy.Array
        The derivatives of y at this time. MetaCaster gives y_delta as a numpy array of zeros to which this function adds the derivatives to.
    parameters : dict {str: Number or callable}
        A dictionary of parameter values or callables used to calculate parameter values at this time point.
    subpop_suffix : string
        This string is of the form '_['coodinate_1,coordinate_2,....']' and is appended to a string denoting parameter specifcally applied to this subpopulation. Alternatively a coordinates argument can be given.
    states_index : dict {str:int}
        This dictionary is used to look up the indexes on y and y_delta for states is this subpopulation.
    foi : float
        Force of infection (lambda) experienced be susceptible hosts in this subpopulation. Note the term lambda could not be used as it is used for lambda functions within python. Therefore, the term foi (Force Of Infection) is used.

    Returns
    -------
    y_deltas : numpy.Array
        Derivatives of variables in disease X's subpopulation model.

    """
    infections = foi * y[states_index['S']]
    progression_from_exposed = parameters['sigma'] * y[states_index['E']]
    probability_of_hospitalisation = parameters['p' + subpop_suffix] # this is our subpopulation specific parameter
    progression_from_infectious = y[states_index['I']]*parameters['gamma']
    recovery = progression_from_infectious*(1-probability_of_hospitalisation)
    hospitalisation = progression_from_infectious*probability_of_hospitalisation
    hospital_recovery = y[states_index['H']]*parameters['eta']


    # Updating y_deltas with derivative calculations from this subpopulation.
    y_deltas[states_index['S']] += - infections
    y_deltas[states_index['E']] += infections - progression_from_exposed
    y_deltas[states_index['I']] += progression_from_exposed - progression_from_infectious
    y_deltas[states_index['H']] += hospitalisation - hospital_recovery
    y_deltas[states_index['R']] += recovery+hospital_recovery
    y_deltas[-2] += hospitalisation - hospital_recovery # The last few elements of y_delta can be used for observed states such Total hospital incidence.
    y_deltas[-1] += hospitalisation #or Total hospitalisations.

    return y_deltas

### 1.1.2 Define metapopulation dimensions.

In this first example we are assuming structure comes from just the risk of hospitalisation (low and high). As we are dealing with a one dimension axis metapopulation we can provide a set of strings as our dimensions (a list/tuple of unique strings is also acceptable). **Note** if dealing with a multidimensional metapopulation the dimensions would have to be a list/tuple of sets of strings.

In [3]:
risk_groups = ['low', 'high']

### 1.1.3 Intialise MetaCaster.

In [4]:
metapop_model = MetaCaster(dimensions=risk_groups,
                           subpop_model=subpop_model,
                           states =['S', 'E', 'I', 'H', 'R'], # States of our model
                           infected_states= ['E', 'I', 'H'], # Infected states of our model, this is different from infectious states.
                           infectious_states=['I'], # Infectious states of our model. These will be involved in force of infection calculations.
                           symptomatic_states=['I', 'H'], # List symptomatic states
                           observed_states=['H','H_cumulative'], # observed_states is where we name what is being tracked in the last few elements of y_deltas.
                           universal_params=['sigma', 'gamma', 'eta'], # These are parameters that are not specific to subpopulations.
                           subpop_params=['p'], # These are parameter that are specific to subpopulation, but not those included in transmission (i.e. beta and rho).
                           foi_population_focus=None # Determines the population denominator used for calculating force of infection see Keeling & Rohani,(2008, pp. 237–240).
                           # If not given or None density dependent transmission is assumed.
                           )

**Note** if you are familiar with python you could alternatively initialise your own subclass of `MetaCaster` with just the dimensions argument. You would need to replace that subclass's subpop_model method and give it the attributes seen in other_model_attributes.

# 1.2 Running model
## 1.2.1 Setting population

For this we will need numpy.

In [5]:
import numpy as np

We need the state_index dictionary for each subpopulation. This gives us our indexes to use on the numpy array defining our initial population $y$.

In [6]:
state_index_dict_high = metapop_model.state_index['high']
state_index_dict_low = metapop_model.state_index['low']
display(state_index_dict_low, state_index_dict_high)

Now we can define $y$.

In [7]:
N = 1e6
proportion_high_risk = 0.1
low_risk_population = N*(1-proportion_high_risk)
high_risk_population = N*proportion_high_risk
y = np.zeros(metapop_model.total_states) # Use the total number of states in our model to define y.
y[state_index_dict_low['S']] = low_risk_population-1
y[state_index_dict_low['I']] = 1
y[state_index_dict_high['S']] = high_risk_population
y

### 1.2.2 Setting up parameters

For this we will need to know the names of all of our parameters:

In [8]:
metapop_model.parameter_names

Now we can define our parameters in a dictionary.

In [9]:
non_subpop_parameters = {'eta': 1/5, 'gamma': 1/7,  'sigma': 1/3}
p_high = 0.3
p_low = 0.01
hospitalisation_probs = {'p_[high]': p_high, 'p_[low]': p_low}
beta = (2/7)/N
beta_parameters = {'beta'+ subpop_suffix: beta for subpop_suffix in metapop_model.subpop_suffixes}
rho = 1
interaction_parameters = {'rho'+ subpop_suffix_i+subpop_suffix_j: rho
                          for subpop_suffix_i in metapop_model.subpop_suffixes
                          for subpop_suffix_j in metapop_model.subpop_suffixes}
parameters = non_subpop_parameters | hospitalisation_probs | beta_parameters | interaction_parameters
parameters

We can now use our parameter dictionary to assign our parameter values.

In [10]:
metapop_model.parameters = parameters
metapop_model.parameters

### 1.2.3 Run metapopulation model

Before running our model we need to define our timeframe $t$.

In [11]:
#set time
end_day = 90
time_step = 1
t = np.arange(0, end_day+time_step, time_step)
t

Now we can use $y$ and $t$ with the `integrate` method to simulate our model.

In [12]:
results_1 = metapop_model.integrate(y,t)
results_1

### 1.2.4 Plotting hospitalisations

Let's plot some of the results of our simulation. For this we will need seaborn and pandas.

In [13]:
import seaborn as sns
import pandas as pd

The function below unstacks our reslults so they can be used by seaborn and then plots the number or hospitalisation ($H$) in our subpopulations and observed_states.

In [14]:
def unstack_results_and_plot_hospitalised(results):
    results_unstacked = results.unstack()
    results_unstacked = results_unstacked.reset_index()
    results_unstacked.columns = ['Subpopulation', 'State','t', 'value']
    hospitalised_results = results_unstacked[results_unstacked['State']=='H']
    sns.lineplot(hospitalised_results,x='t',y='value',hue='Subpopulation')
    return results_unstacked

In [15]:
results_1_unstacked = unstack_results_and_plot_hospitalised(results_1)

We can find out the peak hospitilisations and total_hospitalisations using the code below.

In [16]:
hospitalised_records = [
    {'Scenario': 'Single infection at start',
     'Total hospitalisations': results_1.loc[90,('observed_states','H_cumulative')],
     'Peak hospitalisations': max(results_1.loc[:,('observed_states','H')]),
     }
]
pd.DataFrame.from_records(hospitalised_records)

# 2. Seeding at X proportion prevalence: MultinomialSeeder

MetaCast comes with a `MultinomialSeeder` for seeding infected population randomly drawing probabilities based time spent in compartments. In this example we will use `MultinomialSeeder` to seed our population with a chosen prevalence before running the model defined in the previous section.

## 2.1 Set up Seeders

For this we need to import metacast `MultinomialSeeder`

In [17]:
from metacast import MultinomialSeeder

Then we need to define a seeding information dictionary for `MultinomialSeeder`. This seeding information is a nested dictionary. The first level keys are the possible branches of your infecteds (infection course). The lowest level keys are the states with values being the rate at which people leave that state.

In [18]:
seeding_info = {'unhospitalised': {'E': 'sigma','I':'gamma'},
                'hospitalised': {'E': 'sigma','I':'gamma', 'H':'eta'},
                }
seeder = MultinomialSeeder(seeding_info)
seeder.set_seed(42) # This class uses random number generation (rng) use set_seed function to set rng seed.

## 2.2 Setup Populations with seeder

We can now random draws for our low and high risk population for a given prevalence.

In [19]:
prevelance = 0.01
low_risk_total_infected = low_risk_population*prevelance
low_risk_infected = seeder.seed_infections(n=low_risk_total_infected,
                                           branch_probability={'unhospitalised': 1- parameters['p_[low]'] ,'hospitalised': parameters['p_[low]']},
                                           parameters=parameters)
low_risk_infected

In [20]:
high_risk_total_infected = high_risk_population*prevelance
high_risk_infected = seeder.seed_infections(n=high_risk_total_infected,
                                            branch_probability={'unhospitalised': 1- parameters['p_[high]'] ,'hospitalised': parameters['p_[high]']},
                                            parameters=parameters)
high_risk_infected

Now we can use those draws in setting up our high and low risk populations.

In [21]:
y =np.zeros(metapop_model.total_states)
low_risk_state_pops = {'S':low_risk_population*(1-prevelance), **low_risk_infected}
for state, index in state_index_dict_low.items():
    if state in low_risk_state_pops:
        y[index] = low_risk_state_pops[state]

y

In [22]:
high_risk_state_pops = {'S':high_risk_population*(1-prevelance), **high_risk_infected}
for state, index in state_index_dict_high.items():
    if state in high_risk_state_pops:
        y[index] = high_risk_state_pops[state]

y[-2] += low_risk_state_pops['H'] + high_risk_state_pops['H']
y

### 2.3 Run metapopulation model

The code below runs the model with our new starting population $y$.

In [23]:
results_seeded = metapop_model.integrate(y,t)
results_seeded

### 2.4 Plotting hospitalisations

In [24]:
unstack_results_and_plot_hospitalised(results_seeded)

Let's compare with Total hospitalised and Peak Hospitalised from previous scenarios.

In [25]:
hospitalised_records.append(
    {'Scenario': 'Starting prevalence of ' + str(prevelance),
     'Total hospitalisations': results_seeded.loc[90,('observed_states','H_cumulative')],
     'Peak hospitalisations': max(results_seeded.loc[:,('observed_states','H')]),
     }
)
pd.DataFrame.from_records(hospitalised_records)

# 3. A Rapid Pathogen test: Discrete Event simulation within a 2 dimensional/axis disease X model.

In this scenario we are going to assume that a new affordable rapid test for disease X has become available. Positive tests result in those in states $E$ or $I$ being moved to an isolating test positive population. For this scenario we will need to:
 * Extend our metapopulation dimensions for those who tested postive and negative.
  * Set up a MetaCast's event queue with the new rapid pathogen test as a discrete event.

The equation for the model in this scenario very similar to the previous scenario's. The subscript $q$ has been added to denote those isolating (in quarantine) as the result of a positive test for disease X.
$$
    \frac{\delta S_{r,q}}{\delta t} = -\lambda_{r,q} S_{r,q}
$$
$$
    \frac{\delta E_{r,q}}{\delta t} = \lambda_{r,q} S_{r,q} - \sigma E_{r,q}
$$
$$
    \frac{\delta I_{r,q}}{\delta t} = \sigma E_{r,q} - \gamma I_{r,q}
$$
$$
    \frac{\delta H_{r,q}}{\delta t} = p_{r,q} \gamma I_{r,q} - \eta H_{r,q}
$$
$$
    \frac{\delta R_{r,q}}{\delta t} = (1-p_{r,q}) \gamma I_{r,q} + \eta H_{r,q}
$$
$$
    \lambda_{i} = \beta_{i} \sum_{j} \rho_{ij} I_j
$$


## 3.1 Change Population Structure

Lets start be changing the dimensions of our metapopulation by adding a new dimension for those who tested positive and those who tested negative.

In [26]:
isolation_groups = ['negative','positive']
rapid_pathogen_test_dimensions = [risk_groups,isolation_groups]

In [27]:
metapop_model.dimensions = rapid_pathogen_test_dimensions
metapop_model.dimensions

## 3.2 Set Population

We need to change our starting population as a result of the change in our metapopulation model's dimensions.

In [28]:
y =np.zeros(metapop_model.total_states)
low_risk_state_pops = {'S':low_risk_population*(1-prevelance), **low_risk_infected}
for state, index in metapop_model.state_index[('low', 'negative')].items():
    if state in low_risk_state_pops:
        y[index] = low_risk_state_pops[state]
high_risk_state_pops = {'S':high_risk_population*(1-prevelance), **high_risk_infected}
for state, index in metapop_model.state_index[('high', 'negative')].items():
    if state in high_risk_state_pops:
        y[index] = high_risk_state_pops[state]

y[-2] += low_risk_state_pops['H'] + high_risk_state_pops['H']
y

## 3.3 Set parameters

We also need to change our parameter values as a result of the change in our metapopulation model's dimensions.

In [29]:
metapop_model.parameter_names

In [30]:
beta_parameters = {'beta' + subpop_suffix: beta for subpop_suffix in metapop_model.subpop_suffixes}
isolation_hospitalisation_probs = {
    **{'p_[high,'+ isolation_group +']': hospitalisation_probs['p_[high]'] for isolation_group in isolation_groups},
    **{'p_[low,'+ isolation_group +']': hospitalisation_probs['p_[low]'] for isolation_group in isolation_groups}
}

These new parameters need to account of the effect of isolation on the strength of transmission between populations $\rho_{ij}$.

In [31]:
transmission_reduction_from_isolation = 0.6
interaction_parameters = {
    **{'rho'+ subpop_suffix_i+'_['+risk_group +',negative]': rho
       for subpop_suffix_i in metapop_model.subpop_suffixes
       for risk_group in risk_groups},
    **{'rho'+ subpop_suffix_i+'_['+risk_group +',positive]': rho*(1-transmission_reduction_from_isolation)
       for subpop_suffix_i in metapop_model.subpop_suffixes
       for risk_group in risk_groups}
}
interaction_parameters

In [32]:
parameters = non_subpop_parameters | isolation_hospitalisation_probs | beta_parameters | interaction_parameters
metapop_model.parameters = parameters

## 3.4 Set up Event Que

First we need to import the `EventQueue` and `TransferEvent`

In [33]:
from metacast.event_handling import EventQueue, TransferEvent

For those that would isolate if they comply with isolation order on a positive test we need the following:
 * The indexes of their current compartment (a from_index).
  * The index of the corresponding isolation compartment they would move to (a to_index).

In [34]:
from_index = [index
              for risk_group in risk_groups
              for state, index in metapop_model.state_index[(risk_group,'negative')].items()
              if state in ['E','I']]
to_index = [index
            for risk_group in risk_groups
            for state, index in metapop_model.state_index[(risk_group,'positive')].items()
            if state in ['E','I']]
display({'from index': from_index,'to index': to_index})

With our from_index and to_index we can set up a `TransferEvent` as our 'Rapid Pathogen Test'. The TransferEvent will also need:
 * A range of time at which to be triggered.
  * The proportion of population to move between the compartments in our from_index and to_index.
* A name.

In [35]:
test_every_x_days = 7
compliance = 0.8
test_sensitivity = 0.6
rapid_pathogen_test_event = TransferEvent(name='Rapid Pathogen Test',
                                          times= range(0,end_day+time_step,test_every_x_days),
                                          proportion= test_sensitivity*compliance,
                                          from_index= from_index,
                                          to_index= to_index)

With our 'Rapid Pathogen Test' set up we can set up our event queue.

In [36]:
testing_eventqueue = EventQueue(rapid_pathogen_test_event)

**Note** you can set up the `EventQueue` with multiple event using a list. There are also more event types available than just the TransferEvent see [https://metacast.readthedocs.io/en/latest/autoapi/metacast/event_handling/events/index.html](https://metacast.readthedocs.io/en/latest/autoapi/metacast/event_handling/events/index.html).

## 3.5 Run simulations with Events

In order to run simulations using the `EventQueue` we need to give the `run_sumulation` method the following arguments:
* model_object : The object used to define and simulate model.
* run_attribute : The name of model_objects method that simulates model. Must return either a numpy array of pandas.DataFrame.
* y0 : A numpy.array storing the intial values of state varibles.
* end_time : The stop time of simulations.
* parameters_attribute : The name of the attribute of model_object that sets parameters (must accept dictionary where keys are strings and values are floats/ints).
* parameters : A dictionary {str : floats/ints} defining the model parameters.

Start_time and simulation_step (simulation time step) are optional. If not given the will be set at default values (start_time=0 and simulation_step=1).

In [37]:
results_rapid_test, transfer_df = testing_eventqueue.run_simulation(model_object=metapop_model,
                                                                    run_attribute='integrate',
                                                                    parameters=parameters,
                                                                    parameters_attribute='parameters',
                                                                    y0=y,
                                                                    end_time=end_day,
                                                                    start_time=0,
                                                                    simulation_step=time_step)
results_rapid_test

As well as a pandas.DataFrame giving results of the simulation the `run_simulation` method return a transfer_df DataFrame outlineing the details of events that transferred populations between compartments. This can be handy in debugging or scenario analyses.

In [38]:
transfer_df

### 3.6 Plotting hospitalisations

In [39]:
results_rapid_test_unstacked = unstack_results_and_plot_hospitalised(results_rapid_test)

Let's compare with Total hospitalised and Peak Hospitalised from previous scenarios.

In [40]:
hospitalised_records.append(
    {'Scenario': 'Tests every ' + str(test_every_x_days) + ' day resulting in ' + str(compliance*test_sensitivity) + ' in isolation ('+
                 str(1-transmission_reduction_from_isolation) + ' transmission).',
     'Total hospitalisations': results_rapid_test.loc[90,('observed_states','H_cumulative')],
     'Peak hospitalisations': max(results_rapid_test.loc[:,('observed_states','H')])
     }
)
pd.DataFrame.from_records(hospitalised_records)

## 3.6 Comparing with Rapid test event turned to null (do nothing) event.

In testing the code `MetaCast` was developed from [(Grunnill et al. 2024)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1011018) we found that the precision of Scipy's `odeint` could vary when running many events in an event queue. Scipy `odeint` being the underlying function for `MetaCaster`'s integrate function. Therefore, we added an easy way to make events do nothing, a `make_event_a_nullevent` method, to check for differences due to odeint differing precision.

Let check that such differences `odeint`'s precision have not crept. To do this we will compare event queue simulation with `rapid_pathogen_test_event` set to do nothing with simulations from section 2.

**Note** it is strongly recommended that make events null events by the event queue not the event itself.

In [41]:
testing_eventqueue.make_events_nullevents(['Rapid Pathogen Test'])
results_rapid_test_null, transfer_df = testing_eventqueue.run_simulation(model_object=metapop_model,
                                                                         run_attribute='integrate',
                                                                         parameters=parameters,
                                                                         parameters_attribute='parameters',
                                                                         y0=y,
                                                                         end_time=end_day,
                                                                         start_time=0,
                                                                         simulation_step=time_step)
results_rapid_test_null

In [42]:
transfer_df

In [43]:
results_rapid_test_null_unstacked = unstack_results_and_plot_hospitalised(results_rapid_test_null)

Let's compare with Total hospitalised and Peak Hospitalised from previous scenarios.

In [44]:
hospitalised_records.append(
    {'Scenario': 'Do nothing event.',
     'Total hospitalisations': results_rapid_test_null.loc[90,('observed_states','H_cumulative')],
     'Peak hospitalisations': max(results_rapid_test_null.loc[:,('observed_states','H')])
     }
)
pd.DataFrame.from_records(hospitalised_records)

As we can see there is no differences caused by a change in odeint's precision. If there were we would have to compare our 'Tests every 7 day resulting in 0.48 in isolation (0.4 transmission)' scenario to the 'Do nothing event'.

# 4. New Vaccine: Multi Dimension/Axis disease X model with flows between some subpopulations.

In this scenario we will be looking at the deployment of a new vaccine that has become available for disease X. This new vaccine is not completely transmission blocking, but does at least provide some protection against hospitalisations in breakthrough infections. We will therefore be replacing the second dimension in our previous model from 'negative' and 'positive' to 'unvaccinated', 'vaccination_lag' and 'vaccinated'. We will also setup flows of population between these dimensions 'unvaccinated'->'vaccination_lag'->'vaccinated'.

The equation for the model in this scenario have changed a bit from the previous scenario's. The subscript $v$ replaces $q$ to denote 'unvaccinated', 'vaccination_lag' and 'vaccinated' groups.

$$
    \frac{\delta S_{r,v}}{\delta t} = \nu_{v-1} S_{r,v-1} - \lambda_{r,v} S_{r,v} - \nu_{v} S_{r,v}
$$
$$
    \frac{\delta E_{r,v}}{\delta t} = \nu_{v-1} E_{r,v-1} + \lambda_{r,v} S_{r,v} - \sigma E_{r,v} - \nu_{v} E_{r,v}
$$
$$
    \frac{\delta I_{r,v}}{\delta t} = \nu_{v-1} I_{r,v-1} + \sigma E_{r,v} - \gamma I_{r,v} - \nu_{v} I_{r,v}
$$
$$
    \frac{\delta H_{r,v}}{\delta t} = \nu_{v-1} H_{r,v-1} + p_{r,v} \gamma I_{r,v} - \eta H_{r,v} - \nu_{v} H_{r,v}
$$
$$
    \frac{\delta R_{r,v}}{\delta t} = \nu_{v-1} R_{r,v-1} + (1-p_{r,v}) \gamma I_{r,v} + \eta H_{r,v} - \nu_{v} R_{r,v}
$$
$$
    \lambda_{i} = \beta_{i} \sum_{j} \rho_{ij} I_j
$$

For our vaccinated populations $\beta_{r,v=vaccinated}=\beta(1-l_v)$ and $p_{r,v=vaccinated}=p_{r}(1-h_v)$. Were $l_v$ is vaccine's effectiveness in decreasing susceptibility and $h_v$ is the vaccines effectiveness in preventing hospitalizing infections.

## 4.1 Set metapopulation dimensions using a list of subpopulation transfer dictionaries.

With flows of population (unvaccinated'->'vaccination_lag'->'vaccinated') we need to define our metapopulation model dimensions differently than in previous scenarios. This time we will define dimensions using a list transfer dictionaries. These transfer dictionaries outline the flow between one subpopulation and another. Each transfer dictionary should contain:

* from_coordinates:  Subpopulation coordinates from which hosts are leaving. All of these entries should be of the same length.
* to_coordinates:  Subpopulation coordinates from which hosts are leaving. All of these entries should be of the same length and the same length as the from_coordinates entries.
* states: Host states which will transition between subpopulations. Single entry of 'all' value means all the available model states transition between subpopulations. Alternatively a list of specific states can be given.
* parameter : Name given to parameter that is responsible for flow of hosts transferring between subpopulations.

In [45]:
vaccination_groups = ['unvaccinated','vaccination_lag' 'vaccinated']
vaccination_transfers = [{'from_coordinates': (risk_group,'unvaccinated'),
                          'to_coordinates': (risk_group,'vaccination_lag'),
                          'states': 'all', 'parameter': 'nu_unvaccinated'}
                         for risk_group in risk_groups]
vacination_lag_transfers = [{'from_coordinates': (risk_group,'vaccination_lag'),
                             'to_coordinates': (risk_group,'vaccinated'),
                             'states': 'all', 'parameter': 'nu_vaccination_lag'}
                            for risk_group in risk_groups]

vaccination_dimensions = vaccination_transfers + vacination_lag_transfers
vaccination_dimensions

In [46]:
metapop_model.dimensions = vaccination_dimensions
metapop_model.dimensions

## 4.2 Set population

We need to change our starting population as a result of the change in our metapopulation model’s dimensions.

In [47]:
metapop_model.total_states

In [48]:
y = np.zeros(metapop_model.total_states)

for state, index in metapop_model.state_index[('high','unvaccinated')].items():
    if state in high_risk_state_pops:
        y[index] = high_risk_state_pops[state]

for state, index in metapop_model.state_index[('low','unvaccinated')].items():
    if state in low_risk_state_pops:
        y[index] =low_risk_state_pops[state]

y[-2] += low_risk_state_pops['H'] + high_risk_state_pops['H']
y

## 4.3 Set parameters

We also need to change our parameter values as a result of the change in our metapopulation model's dimensions.

In [49]:
metapop_model.parameter_names

This time around we are going to setup a function for setting up the parameters as we will be using the same metapopulation model in the next exercise.

In [50]:
def vaccination_parameters_setup(nu_unvaccinated,
                                 nu_vaccination_lag,
                                 l_v,
                                 h_v,
                                 other_parameters,
                                 metapop_model):
    parameters = {key: value for key, value in other_parameters.items() if key in metapop_model.parameter_names}
    parameters.update({'nu_unvaccinated': nu_unvaccinated,
                       'nu_vaccination_lag': nu_vaccination_lag})

    beta = other_parameters['beta']
    parameters.update({'beta_[high,unvaccinated]': beta,
                       'beta_[high,vaccinated]': beta*(1-l_v),
                       'beta_[high,vaccination_lag]': beta,
                       'beta_[low,unvaccinated]': beta,
                       'beta_[low,vaccinated]': beta*(1-l_v),
                       'beta_[low,vaccination_lag]': beta})

    if h_v < l_v:
        raise ValueError('h_v must be greater than or equal to l_v. ' +
                         'Otherwise vaccine reduced severity given reduced susceptibility is negative.')

    vaccine_reduced_severity_given_reduced_susceptibility = 1-((1-h_v)/(1-l_v))
    p_high = other_parameters['p_[high]']
    p_low = other_parameters['p_[low]']
    parameters.update({'p_[high,unvaccinated]': p_high,
                       'p_[high,vaccinated]': p_high*(1-vaccine_reduced_severity_given_reduced_susceptibility),
                       'p_[high,vaccination_lag]': p_high,
                       'p_[low,unvaccinated]': p_low,
                       'p_[low,vaccinated]': p_low*(1-vaccine_reduced_severity_given_reduced_susceptibility),
                       'p_[low,vaccination_lag]': p_low})
    rho = other_parameters['rho']
    parameters.update({'rho'+ subpop_suffix_i+subpop_suffix_j: rho
                       for subpop_suffix_i in metapop_model.subpop_suffixes
                       for subpop_suffix_j in metapop_model.subpop_suffixes})
    return parameters

def prob_over_many_days_to_prob_on_a_day(prob, many_days):
    return 1-(1-prob)**(1/many_days)

In [51]:
v_day = end_day
prob_vaccinated_by_v_day = 0.7
prob_vaccinated_per_day = prob_over_many_days_to_prob_on_a_day(prob_vaccinated_by_v_day ,v_day)
vaccination_lag = 1/14
vaccine_reduced_susceptibility = 0.4
vaccine_reduced_severity = 0.8
other_parameters = {'beta':beta, 'rho': rho, **non_subpop_parameters, **hospitalisation_probs}
parameters = vaccination_parameters_setup(nu_unvaccinated=prob_vaccinated_per_day*compliance,
                                          nu_vaccination_lag=vaccination_lag,
                                          l_v=vaccine_reduced_susceptibility,
                                          h_v=vaccine_reduced_severity,
                                          other_parameters=other_parameters,
                                          metapop_model=metapop_model)
parameters

In [52]:
metapop_model.parameters = parameters

### 4.4 Run metapopulation model

Lets run our model.

In [53]:
results_vaccination = metapop_model.integrate(y,t)
results_vaccination

### 4.5 Plotting hospitalisations

In [54]:
results_vaccination_unstacked = unstack_results_and_plot_hospitalised(results_vaccination)

Let's compare with Total hospitalised and Peak Hospitalised from previous scenarios.

In [55]:
hospitalised_records.append(
    {'Scenario': 'Vaccinating ' + str(prob_vaccinated_by_v_day) + ' population by day '+ str(v_day),
     'Total hospitalisations': results_vaccination.loc[90,('observed_states','H_cumulative')],
     'Peak hospitalisations': max(results_vaccination.loc[:,('observed_states','H')])
     }
)
hospitalised_results_df = pd.DataFrame.from_records(hospitalised_records)
hospitalised_results_df

In [56]:
metapop_model.get_state_index_dict_of_coordinate('high')

# 5. New Vaccine's Efficacy is Uncertain: Latin Hypercube Sampling in simulation of a 2 dimensional disease X model.

Some uncertainties have come about in the new vaccine for disease X, both in terms of efficacy and delivery. Therefore, in this next exercise we will running a sensitivity analyses of the metapopulation model from the previous scenario using Latin Hypercube Sampling. For a description of Latin Hypercube Sampling (LHS) and Partial Rank Correlation Coefficients I suggest you read [Marino et, al. (2008)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2570191/).

For generating an LH sample, running simulations for all of the parameter sets in the LH sample we will need to import MetaCast's lhs_prcc function.

In [57]:
from metacast.sensitivity_analyses import lhs_prcc

## 5.1 Setup a function for running the metapopulation model.

lhs_prcc in tangent with a user defined function that sets up the model and runs a sampled parameter set. Key features of this user defined function are:
* The first argument of this function must be for a dictionary of the sampled parameters.
* The returned object must be a dictionary combining the first argument of input parameters with the results you wished to see from the simulation.

**Note** we will see later that lhs_prcc can pass other key word arguments (kwargs) to this user defined function.

Lets create such a function for running our vaccination model below.

In [58]:
def vaccination_lhs_sim(sample, # First argument must be for a dictionary of the sampled parameters
                        t,
                        y0,
                        fixed_parameter,
                        metapop_model):
    parameters = vaccination_parameters_setup(**sample, other_parameters=fixed_parameter,
                                              metapop_model=metapop_model)
    metapop_model.parameters = parameters
    results = metapop_model.integrate(y0, t)
    focused_results = {'Total hospitalisations': results.loc[90, ('observed_states', 'H_cumulative')],
                       'Peak hospitalisations': max(results.loc[:, ('observed_states', 'H')])}
    results_and_sample = sample | focused_results #
    return results_and_sample

## 5.2 Setup parameter ranges to sample from.

In order to generate a LH sample `lhs_prcc` requires a pandas.DataFrame outlining the boundaries of LHS (our parameter certainties). **Note** currently metacast only support latin hypercube sampling with uniform distributions. Key features of this DataFrame:
* Must contain fields 'Lower Bound' and 'Upper Bound'.
* The name of the parameters is assumed to be in the index of the DataFrame.

The code below creates such a DataFrame outlining certainties surrounding disease X's vaccination parameters.

In [59]:
parameter_samples_records = [
    {'parameter': 'nu_unvaccinated',
     'Lower Bound': prob_over_many_days_to_prob_on_a_day(0.5, v_day),
     'Upper Bound': prob_over_many_days_to_prob_on_a_day(0.95, v_day)}, # Rate of vaccinations.

    {'parameter': 'nu_vaccination_lag', 'Lower Bound': 1/28, 'Upper Bound': 1/7}, # Lag between vaccination and vaccination being effective.
    {'parameter': 'l_v', 'Lower Bound': 0.3, 'Upper Bound': 0.6}, # Reduction in susceptibility to disease X after vaccination.
    {'parameter': 'h_v', 'Lower Bound': 0.7, 'Upper Bound': 0.95} # Reduction in hospitalisation with disease X after vaccination.
]
parameter_samples_df = pd.DataFrame.from_records(parameter_samples_records)
parameter_samples_df.set_index('parameter',inplace=True) # The parameters must be set as the index of the dataframe.
parameter_samples_df

##  5.3 Run simulations with LHS

 Let's test run `lhs_prcc` with a small sample size. As well as the DataFrame outlineing the sampling of parameters and our user defined function for running our model, `lhs_prcc` requires the number of samples generated in the LHS. All other arguments in the code cell below are passed to the function for running our model.

In [60]:
results_and_sample_df, prccs = lhs_prcc(parameters_df=parameter_samples_df,
                                        sample_size=10,
                                        model_run_method=vaccination_lhs_sim,
                                        metapop_model=metapop_model,
                                        t=t,
                                        y0=y,
                                        fixed_parameter=other_parameters)
results_and_sample_df

As well as a dataframe of results and sampled parameters `lhs_prcc` performs analyses the ranked correlation between parameters and results. This form of rank correlation coefficent adjusts for the effect of other paramters [(Marino et, al. 2008)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2570191/). The results partial rank correlation coefficient analyses are pictured below.

In [61]:
prccs

Our initial sample size is quite small. We may need to run a much larger sample size. To do so in serially would take a long time. Luckily `lhs_prcc` can take a `dask.distributed.Client` for running simulations in parallel.

##  5.4 Run simulations with LHS in parallel

First we will need to know the number of cores on the macine we are using. It is best practice to not run use all your machines' cores as this will make running other programs difficult.

In [62]:
import os
number_cpu = os.cpu_count()
number_cpu

Then we need to setup a dask Client.

In [63]:
# setup the dask cluster
import dask

client = dask.distributed.Client(n_workers=number_cpu-1, threads_per_worker=1)
client

Clicking on the hyperlink in the output above will bring up the dask client dashboard. The dashboard will display the dask client's progress with running any tasks given to it. The code using `lhs_prcc` below has an additional argument from before, `client` this is the client you wish to use for parallel simulations.

In [64]:
results_and_sample_df, prccs = lhs_prcc(parameters_df=parameter_samples_df,
                                        sample_size=1000,
                                        model_run_method=vaccination_lhs_sim,
                                        metapop_model=metapop_model,
                                        client= client, # this argument tells lhs_prcc to use this dask client.
                                        t=t,
                                        y0=y,
                                        fixed_parameter=other_parameters)

It is best practice to shutdown (`close`) a dask client once you are finished with it.

In [65]:
client.close()

In [66]:
prccs

## 5.5 Plotting how Effective Vaccination was given Uncertainties.

Lets start by plotting box plots of total hospitalisations and peak hospitalisations from results_and_sample_df.

In [67]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
for index, result in enumerate(['Total hospitalisations', 'Peak hospitalisations']):
    axs[index].violinplot(results_and_sample_df[result],
                          showmeans=False,
                          showmedians=True)
    axs[index].set_title(result)

Now lets start plot % reductions in total hospitalisations and peak hospitalisations due to vaccination.

In [68]:
hospitalised_results_df

In [69]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
for index, result in enumerate(['Total hospitalisations', 'Peak hospitalisations']):
    baseline_value = hospitalised_results_df.loc[hospitalised_results_df['Scenario']=='Starting prevalence of 0.01',result].values[0]
    to_plot = (results_and_sample_df[result] - baseline_value)/baseline_value
    to_plot *= 100
    axs[index].violinplot(to_plot,
                          showmeans=False,
                          showmedians=True)
    axs[index].set_title('% reduction in ' + result)